# Script to help user to define a configuration cell for ODC notebooks

*****

As geographical extent, time period, product and measurements differ between ODC dataset, this scrip explore the connected ODC and help user to create a configuration cell with proper data, to be manually copy/pasted or loaded in ODC Jupyter notebook.

In [ ]:
# Import modules

# reload module before executing code
%load_ext autoreload
%autoreload 2

# define modules locations (you might have to adapt define_mod_locs.py)
%run ../swiss_utils/define_mod_locs.py

import ipywidgets as widgets

from datetime import timedelta
from shapely.geometry import Polygon

from utils.data_cube_utilities.dc_load import get_product_extents

from swiss_utils.data_cube_utilities.sdc_utilities import get_platform, new_get_query_metadata
from swiss_utils.data_cube_utilities.sdc_advutils import draw_map

import datacube
dc = datacube.Datacube()

In [ ]:
# Select the proper format:
# if NetCDF is not available it meand ingestion wasn't performed and use original Geotiff instead

products = dc.list_products()
fmts = set(map(str.upper,set(products.format)))
if 'NETCDF' in fmts:
    format_sel = 'NETCDF'
elif 'GEOTIFF' in fmts:
    format_sel = 'GEOTIFF'
else:
    print('Cannot find proper format (Geotiff or NetCDF) to run the rest os the script !')

In [ ]:
# Select the product

product_names = products[products['format'].str.upper() == format_sel].name
product_sel = widgets.RadioButtons(options=product_names,
                                   disabled=False)
display(widgets.Label('Select a product and run the next cell: '), product_sel)

In [ ]:
# automatically set platform

platform = get_platform(dc, [product_sel.value])[0]
print('platform: {}'.format(platform))

In [ ]:
# Select measurements

measurement_list = dc.list_measurements(with_pandas=False)
measurements_for_product = filter(lambda x: x['product'] == product_sel.value, measurement_list)
valid_measurements_name_array = set(map(lambda x: x['name'], measurements_for_product))
# Landsat and Sentinel 2
try:
    measur_sel = widgets.SelectMultiple(options=sorted(list(valid_measurements_name_array)),
                                        value = ['blue', 'green', 'red'],
                                        disabled=False)
# Sentinel 1
except:
    measur_sel = widgets.SelectMultiple(options=sorted(list(valid_measurements_name_array)),
                                        disabled=False)
display(widgets.Label('Select a measurements and run the next cell: '), measur_sel)

In [ ]:
# Convert tupple to list

measur_sel = list(measur_sel.value)

In [ ]:
import utils.data_cube_utilities.data_access_api as dc_api  
api = dc_api.DataAccessApi()

mtd = api.get_query_metadata(product = product_sel.value, measurements=[], output_crs = "epsg:4326", resolution = (-0.1,0.1))

full_lat = mtd['lat_extents']
full_lon = mtd['lon_extents']
min_max_dates = mtd['time_extents']

# Add an empty map you can draw on it
m, poly = draw_map(full_lat, full_lon)
print('Zoom, pan and draw a rectangle or polygon (the bounding box will be used) and run the next cell:')
m

In [ ]:
# Once a feature is drawn, extract the bounding box of the last feature drawn

coords = poly.last_draw['geometry']['coordinates']
geo_extent = Polygon(coords[0]).bounds

min_lon = geo_extent[0]
min_lat = geo_extent[1]
max_lon = geo_extent[2]
max_lat = geo_extent[3]

In [ ]:
# Select time period

start_date = widgets.DatePicker(description='Start date',
                                value = min_max_dates[0].date(),
                                disabled=False)
end_date = widgets.DatePicker(description='End date',
                              value = min_max_dates[1].date(),
                              disabled=False)
display(widgets.Label('Define time period (cannot be outside of the initial displaye time) and run the next cell:'),
        widgets.HBox([start_date, end_date]))
print('End date is not inclusive:\nfirst day of a month will only select a dataset till the last day of previous month')

In [ ]:
# check defined time period

assert start_date.value >= min_max_dates[0].date(), \
       'Start date cannot be defined before {}'.format(min_max_dates[0].date())
assert end_date.value <= min_max_dates[1].date(), \
       'End date cannot be defined after {}'.format(min_max_dates[1].date())
assert start_date.value < end_date.value, \
       'End date is defined before start date'

end_date.value += timedelta(days=1) # end_date is not inclusive !

print('Time period is OK')

In [ ]:
# Resume configuration parameters in a format ready to be copy/pasted to a new cell,
# and in a txt file to be loaded with the '%load config_cell.txt' magic.

str = '''# Configuration

platform = '{}'
product = '{}'
measurements = {}

min_lon = {}
max_lon = {}
min_lat = {}
max_lat = {}

start_date = datetime.strptime('{}', '%Y-%m-%d')
end_date = datetime.strptime('{}', '%Y-%m-%d')'''.\
format(get_platform(dc, [product_sel.value])[0],
       product_sel.value,
       measur_sel,
       min_lon, max_lon, min_lat, max_lat,
       start_date.value.strftime('%Y-%m-%d'), end_date.value.strftime('%Y-%m-%d'))
print(str)
with open('config_cell.txt', 'w') as text_file:
    print(str, file=text_file)